# Hard Coding a MET File

This notebook copies radn (solar radiation), evap (evaporation), vp (vapor pressure), and code values from York16_test.met into the Anameka MET files.


## 1. Imports and Configuration


In [ ]:
import pandas as pd
import numpy as np
import os
import re
from pathlib import Path

# File paths
SOURCE_MET_FILE = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\York16_test.met"
TARGET_MET_FILES = [
    r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka South_ACCESS CM2_SSP585.met",
    r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka South_ACCESS CM2_SSP245.met"
]

print("Configuration:")
print(f"  Source file: {os.path.basename(SOURCE_MET_FILE)}")
print(f"  Target files: {[os.path.basename(f) for f in TARGET_MET_FILES]}")


## 2. Read radn, evap, vp, and code values from York16_test.met


In [ ]:
def parse_met_file(filepath):
    """Parse MET file and return header lines and data as list of dictionaries."""
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # Find data start
    data_start_idx = None
    header_lines = []
    
    for i, line in enumerate(lines):
        if 'year  day radn' in line.lower():
            header_lines = lines[:i+2]
            data_start_idx = i + 2
            break
    
    if data_start_idx is None:
        raise ValueError(f"Could not find data section in {filepath}")
    
    data_rows = []
    for line in lines[data_start_idx:]:
        line = line.strip()
        if not line or line.startswith('!'):
            continue
        
        # Parse - handle empty radn field
        parts = line.split()
        if len(parts) >= 5 and parts[0].isdigit():
            try:
                year = int(parts[0])
                day = int(parts[1])
                
                # Determine if radn is present (6+ parts) or missing (5 parts)
                if len(parts) >= 6:
                    try:
                        potential_radn = float(parts[2])
                        if 0 <= potential_radn <= 50:
                            radn = parts[2]
                            maxt = float(parts[3])
                            mint = float(parts[4])
                            rain = float(parts[5])
                            evap = parts[6] if len(parts) > 6 else ''
                            vp = parts[7] if len(parts) > 7 else ''
                            code = parts[8] if len(parts) > 8 else ''
                        else:
                            radn = ''
                            maxt = float(parts[2])
                            mint = float(parts[3])
                            rain = float(parts[4])
                            evap = parts[5] if len(parts) > 5 else ''
                            vp = parts[6] if len(parts) > 6 else ''
                            code = parts[7] if len(parts) > 7 else ''
                    except ValueError:
                        radn = ''
                        maxt = float(parts[2])
                        mint = float(parts[3])
                        rain = float(parts[4])
                        evap = parts[5] if len(parts) > 5 else ''
                        vp = parts[6] if len(parts) > 6 else ''
                        code = parts[7] if len(parts) > 7 else ''
                else:
                    radn = ''
                    maxt = float(parts[2])
                    mint = float(parts[3])
                    rain = float(parts[4])
                    evap = ''
                    vp = ''
                    code = ''
                
                data_rows.append({
                    'year': year,
                    'day': day,
                    'radn': radn,
                    'maxt': maxt,
                    'mint': mint,
                    'rain': rain,
                    'evap': evap,
                    'vp': vp,
                    'code': code
                })
            except (ValueError, IndexError):
                continue
    
    return header_lines, data_rows

# Read source file
print("Reading source file...")
source_header, source_data = parse_met_file(SOURCE_MET_FILE)
source_df = pd.DataFrame(source_data)
source_df['radn_float'] = pd.to_numeric(source_df['radn'], errors='coerce')
source_df['evap_float'] = pd.to_numeric(source_df['evap'], errors='coerce')
source_df['vp_float'] = pd.to_numeric(source_df['vp'], errors='coerce')

print(f"  Loaded {len(source_df)} rows from {os.path.basename(SOURCE_MET_FILE)}")
print(f"  Year range: {source_df['year'].min()} to {source_df['year'].max()}")
print(f"  Sample values:")
print(source_df[['year', 'day', 'radn', 'evap', 'vp', 'code']].head(10))


## 3. Function to update MET files with radn, evap, vp, and code values


In [ ]:
def update_met_file_with_columns(target_filepath, source_df):
    """
    Update MET file with radn, evap, vp, and code values from source DataFrame.
    For missing years, repeat the last 5 years of data.
    """
    print(f"\n  Processing: {os.path.basename(target_filepath)}")
    
    # Read target file
    target_header, target_data = parse_met_file(target_filepath)
    target_df = pd.DataFrame(target_data)
    
    print(f"    Target year range: {target_df['year'].min()} to {target_df['year'].max()}")
    print(f"    Target rows: {len(target_df)}")
    
    # Get unique years from source
    source_years = sorted(source_df['year'].unique())
    print(f"    Source year range: {source_years[0]} to {source_years[-1]} ({len(source_years)} years)")
    
    # Create lookup dictionaries: (year, day) -> value
    radn_lookup = {}
    evap_lookup = {}
    vp_lookup = {}
    code_lookup = {}
    
    for _, row in source_df.iterrows():
        key = (row['year'], row['day'])
        if pd.notna(row['radn_float']):
            radn_lookup[key] = row['radn_float']
        if row['evap'] and row['evap'].strip():
            try:
                evap_lookup[key] = float(row['evap'])
            except (ValueError, TypeError):
                evap_lookup[key] = row['evap']  # Keep as string
        if row['vp'] and row['vp'].strip():
            try:
                vp_lookup[key] = float(row['vp'])
            except (ValueError, TypeError):
                vp_lookup[key] = row['vp']  # Keep as string
        if row['code'] and row['code'].strip():
            code_lookup[key] = row['code']
    
    # Determine which years need data
    target_years = sorted(target_df['year'].unique())
    missing_years = [y for y in target_years if y not in source_years]
    
    if missing_years:
        print(f"    Missing years: {missing_years[:5]}... (total: {len(missing_years)})")
        # Use last 5 years from source
        last_5_years = source_years[-5:]
        print(f"    Repeating last 5 years: {last_5_years}")
        
        # Create mapping for missing years - repeat the last 5 years (cycling)
        for i, missing_year in enumerate(missing_years):
            source_year = last_5_years[i % 5]
            source_year_data = source_df[source_df['year'] == source_year]
            for _, row in source_year_data.iterrows():
                key = (missing_year, row['day'])
                if pd.notna(row['radn_float']):
                    radn_lookup[key] = row['radn_float']
                if row['evap'] and row['evap'].strip():
                    try:
                        evap_lookup[key] = float(row['evap'])
                    except (ValueError, TypeError):
                        evap_lookup[key] = row['evap']
                if row['vp'] and row['vp'].strip():
                    try:
                        vp_lookup[key] = float(row['vp'])
                    except (ValueError, TypeError):
                        vp_lookup[key] = row['vp']
                if row['code'] and row['code'].strip():
                    code_lookup[key] = row['code']
    
    # Helper function to find value for a missing key
    def find_fallback_value(lookup_dict, year, day, source_years, last_5_years):
        """Find a fallback value for missing (year, day) by checking previous years or cycling through last 5 years."""
        # Try same day in previous years
        for prev_year in range(year - 1, source_years[0] - 1, -1):
            if (prev_year, day) in lookup_dict:
                return lookup_dict[(prev_year, day)]
        # Try day 365 if looking for day 366 (leap year handling)
        if day == 366:
            for prev_year in range(year - 1, source_years[0] - 1, -1):
                if (prev_year, 365) in lookup_dict:
                    return lookup_dict[(prev_year, 365)]
        # Use last 5 years pattern
        year_idx = (year - source_years[0]) % 5
        source_year = last_5_years[year_idx]
        # Try same day
        if (source_year, day) in lookup_dict:
            return lookup_dict[(source_year, day)]
        # Try day 365 for day 366
        if day == 366 and (source_year, 365) in lookup_dict:
            return lookup_dict[(source_year, 365)]
        # Last resort: use any available value for this day
        for any_year in reversed(source_years):
            if (any_year, day) in lookup_dict:
                return lookup_dict[(any_year, day)]
            if day == 366 and (any_year, 365) in lookup_dict:
                return lookup_dict[(any_year, 365)]
        return None
    
    # Update target DataFrame with all values
    radn_count = 0
    evap_count = 0
    vp_count = 0
    code_count = 0
    
    # Build last_5_years for fallback
    last_5_years = source_years[-5:] if len(source_years) >= 5 else source_years
    
    for idx, row in target_df.iterrows():
        key = (row['year'], row['day'])
        year = row['year']
        day = row['day']
        
        # Update radn
        if key in radn_lookup:
            target_df.at[idx, 'radn'] = f"{radn_lookup[key]:.1f}"
            radn_count += 1
        else:
            # Fill missing radn
            fallback = find_fallback_value(radn_lookup, year, day, source_years, last_5_years)
            if fallback is not None:
                target_df.at[idx, 'radn'] = f"{fallback:.1f}"
                radn_count += 1
        
        # Update evap
        if key in evap_lookup:
            if isinstance(evap_lookup[key], float):
                target_df.at[idx, 'evap'] = f"{evap_lookup[key]:.1f}"
            else:
                target_df.at[idx, 'evap'] = str(evap_lookup[key])
            evap_count += 1
        else:
            # Fill missing evap
            fallback = find_fallback_value(evap_lookup, year, day, source_years, last_5_years)
            if fallback is not None:
                if isinstance(fallback, float):
                    target_df.at[idx, 'evap'] = f"{fallback:.1f}"
                else:
                    target_df.at[idx, 'evap'] = str(fallback)
                evap_count += 1
        
        # Update vp
        if key in vp_lookup:
            if isinstance(vp_lookup[key], float):
                target_df.at[idx, 'vp'] = f"{vp_lookup[key]:.1f}"
            else:
                target_df.at[idx, 'vp'] = str(vp_lookup[key])
            vp_count += 1
        else:
            # Fill missing vp
            fallback = find_fallback_value(vp_lookup, year, day, source_years, last_5_years)
            if fallback is not None:
                if isinstance(fallback, float):
                    target_df.at[idx, 'vp'] = f"{fallback:.1f}"
                else:
                    target_df.at[idx, 'vp'] = str(fallback)
                vp_count += 1
        
        # Update code
        if key in code_lookup:
            target_df.at[idx, 'code'] = str(code_lookup[key])
            code_count += 1
        else:
            # Fill missing code
            fallback = find_fallback_value(code_lookup, year, day, source_years, last_5_years)
            if fallback is not None:
                target_df.at[idx, 'code'] = str(fallback)
                code_count += 1
    
    print(f"    Updated: {radn_count} radn, {evap_count} evap, {vp_count} vp, {code_count} code values")
    
    # Write updated MET file
    output_filepath = target_filepath
    with open(output_filepath, 'w', encoding='utf-8') as f:
        # Write header
        f.writelines(target_header)
        
        # Write data rows
        for _, row in target_df.iterrows():
            radn_str = row['radn'] if 'radn' in row and row['radn'] and str(row['radn']).strip() else "      "
            evap_str = row['evap'] if 'evap' in row and row['evap'] and str(row['evap']).strip() else "      "
            vp_str = row['vp'] if 'vp' in row and row['vp'] and str(row['vp']).strip() else "      "
            code_str = row['code'] if 'code' in row and row['code'] and str(row['code']).strip() else "      "
            
            line = f"{int(row['year']):4d} {int(row['day']):4d} {radn_str:>6s} {row['maxt']:6.1f} {row['mint']:6.1f} {row['rain']:6.1f} {evap_str:>6s} {vp_str:>6s} {code_str:>6s}\n"
            f.write(line)
    
    print(f"    [OK] File updated: {os.path.basename(output_filepath)}")
    
    return target_df

# Update all target files
print("\n" + "="*70)
print("Updating MET files with radn, evap, vp, and code values")
print("="*70)

for target_file in TARGET_MET_FILES:
    if os.path.exists(target_file):
        updated_df = update_met_file_with_columns(target_file, source_df)
    else:
        print(f"  ERROR: File not found: {target_file}")

print("\n" + "="*70)
print("All files updated successfully!")
print("="*70)
